# Web Scraping with Beautiful Soup: Solutions

In [15]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [16]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

#Se realiza un peticion GET a la pagina 
#Lee el conetendo html de la pagina 
# Convierte el html de la pagina en objetos para facilitar el tratamiento de datos. 

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

## Challenge: Find All

Use Beautiful Soup to find all the `a` elements with class `mainmenu`.

In [ ]:
soup.select("a.mainmenu")

#Se busca elementos html con el selector

## Challenge: Extract Specific Attributes

Extract all `href` attributes for each `mainmenu` URL.

In [ ]:
[link['href'] for link in soup.select("a.mainmenu")]

#Se extrae todos los atributos de elemento a.mainmenu 

## Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format. 

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`. 

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips: 

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

# Create empty list to store our data
#SE CREA UN ARRAY VACIO PARA ALMACENAR LOS DATOS
members = []


# Returns every ‘tr tr tr’ css selector in the page
##SELECCIONAMOS TODAS LAS FILAS DE LA TABLA EN CONTRADA 
rows = soup.select('tr tr tr')
# Get rid of junk rows

#FILTRA LAS FILAS QUE TIENEN LA CALSE TD.DETAIL
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
##LAZO PARA ITERRAR TODAS LAS FILAS 
for row in rows:

    #OBTIENE LOS DATOS 
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    
    # YOUR CODE HERE
    # Extract href
    href = row.select('a')[1]['href']
    # Create full path
    full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"
    
    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [ ]:
#SE OBTENE LOS PRIMEROS 5 ELEMENTOS DE LA TABLA

members[:5]

## Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator. 

In [ ]:
#FUNCION PARA EXTRAER LA INFORMACION DEL SITIO WEB
 #1. Obtencion del html
 #2. Filtrado de información mediante la clase html
 #3. Iteracion de cada fila del css, para obtener la inforamcion de cada Senador
 #4. Almacenamientos de datos 

def get_members(url):
    # Make a GET request
    req = requests.get(url)
    # Read the content of the server’s response
    src = req.text
    # Soup it
    soup = BeautifulSoup(src, "lxml")
    # Create empty list to store our data
    members = []

    # Returns every ‘tr tr tr’ css selector in the page
    rows = soup.select('tr tr tr')
    # Get rid of junk rows
    rows = [row for row in rows if row.select('td.detail')]

    # Loop through all rows
    for row in rows:
        # Select only those 'td' tags with class 'detail'
        detail_cells = row.select('td.detail') 
        # Keep only the text in each of those cells
        row_data = [cell.text for cell in detail_cells]
        # Collect information
        name = row_data[0]
        district = int(row_data[3])
        party = row_data[4]

        # YOUR CODE HERE
        # Extract href
        href = row.select('a')[1]['href']
        # Create full path
        full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"

        # Store in a tuple
        senator = (name, district, party, full_path)
        # Append to list
        members.append(senator)
    return(members)

In [ ]:
# Test your code!

#SE LLAMA LA FUNCION GET_MEMBERS, ANTES DEFINIDA Y SE ESTABLE LA DIRECCION LA WEB
# VERIFICA LA CANTIDAD DE SENADORES EXTRAIDOS.
 
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

## Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [ ]:

def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr tr tr')
    bills = []
    # Iterate over rows
    for row in rows:
        # Grab all bill list cells
        cells = row.select('td.billlist')
        # Keep in mind the name of the senator is not a billlist class!
        if len(cells) == 5:
            row_text = [cell.text for cell in cells]
            # Extract info from row text
            bill_id = row_text[0]
            description = row_text[1]
            chamber = row_text[2]
            last_action = row_text[3]
            last_action_date = row_text[4]
            # Consolidate bill info
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

In [ ]:
test_url = senate_members[0][3]
get_bills(test_url)[0:5]

NameError: name 'senate_members' is not defined

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
bills_dict = {}
for member in senate_members[:5]:
    bills_dict[member[1]] = get_bills(member[3])
    time.sleep(1)

In [ ]:
len(bills_dict[52])